# <center> Домашнее задание №6. Решение
## <center> Исследование OLS, Lasso и Random Forest в задаче регрессии
    
<img src="https://habrastorage.org/webt/-h/ns/aa/-hnsaaifymavmmudwip9imcmk58.jpeg" width=30%>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Мы работаем с датасетом UCI Wine Quality (скачивать не нужно — он уже есть в репозитории).**

In [ ]:
data = pd.read_csv('../data/winequality-white.csv')

In [ ]:
data.head()

In [ ]:
data.info()

**Выделите целевой признак, разделите данные в пропорции 7:3 (30% — отложенная выборка, используйте random_state=17) и предобработайте данные с помощью `StandardScaler`.**

In [ ]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, 
                                                          random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую модель линейной регрессии (метод наименьших квадратов, OLS).**

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train);

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки (MSE) предсказаний модели на обучающей и отложенной выборках?**

In [ ]:
print("Среднеквадратичная ошибка (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Среднеквадратичная ошибка (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

**Отсортируйте признаки по степени их влияния на целевой признак (качество вина). Учтите, что и большие положительные, и большие отрицательные коэффициенты означают сильное влияние на целевую переменную. Удобно использовать `pandas.DataFrame`.**

**<font color='red'>Вопрос 2:</font> Какой признак модель линейной регрессии считает наиболее влиятельным на качество вина?**

In [ ]:
linreg_coef = pd.DataFrame({'coef': linreg.coef_, 'coef_abs': np.abs(linreg.coef_)},
                          index=data.columns.drop('quality'))
linreg_coef.sort_values(by='coef_abs', ascending=False)

## Регрессия Lasso

**Обучите модель LASSO с $\alpha = 0.01$ (слабая регуляризация) на отмасштабированных данных. Установите random_state=17.**

In [ ]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

**Какой признак наименее информативен для предсказания качества вина согласно этой модели LASSO?**

In [ ]:
lasso1_coef = pd.DataFrame({'coef': lasso1.coef_, 'coef_abs': np.abs(lasso1.coef_)},
                          index=data.columns.drop('quality'))
lasso1_coef.sort_values(by='coef_abs', ascending=False)

**Обучите LassoCV с random_state=17, чтобы подобрать лучшее значение $\alpha$ с помощью 5-fold кросс-валидации.**

In [ ]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)

In [ ]:
lasso_cv.alpha_

**<font color='red'>Вопрос 3:</font> Какой признак наименее информативен для предсказания качества вина согласно настроенной модели LASSO?**

In [ ]:
lasso_cv_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'coef_abs': np.abs(lasso_cv.coef_)},
                          index=data.columns.drop('quality'))
lasso_cv_coef.sort_values(by='coef_abs', ascending=False)

**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки (MSE) предсказаний настроенной модели LASSO на обучающей и отложенной выборках?**

In [ ]:
print("Среднеквадратичная ошибка (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Среднеквадратичная ошибка (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

## Random Forest

**Обучите Random Forest с параметрами по умолчанию, задав только random_state=17.**

In [ ]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки (MSE) модели RF на обучающей выборке, при кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [ ]:
print("Среднеквадратичная ошибка (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Среднеквадратичная ошибка (cv): %.3f" % np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, 
                                                                       scoring='neg_mean_squared_error'))))
print("Среднеквадратичная ошибка (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

**Настройте гиперпараметры `max_features` и `max_depth` с помощью GridSearchCV и снова проверьте среднюю MSE при кросс-валидации и MSE на отложенной выборке.**

In [ ]:
forest_params = {'max_depth': list(range(10, 25)), 
                  'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=17), 
                                 forest_params, 
                                 scoring='neg_mean_squared_error',  
                                 n_jobs=-1, cv=5,
                                  verbose=True)
locally_best_forest.fit(X_train_scaled, y_train)

In [ ]:
locally_best_forest.best_params_, locally_best_forest.best_score_

**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки (MSE) настроенной модели RF при кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [ ]:
print("Среднеквадратичная ошибка (cv): %.3f" % np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_,
                                                        X_train_scaled, y_train, 
                                                        scoring='neg_mean_squared_error'))))
print("Среднеквадратичная ошибка (test): %.3f" % mean_squared_error(y_holdout, 
                                                             locally_best_forest.predict(X_holdout_scaled)))

**Выведите важность признаков по модели RF. Удобно представить результат в виде DataFrame.**

**<font color='red'>Вопрос 7:</font> Какой признак является самым важным по версии модели Random Forest?**

In [ ]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, 
                             columns=['coef'], index=data.columns[:-1]) 
rf_importance.sort_values(by='coef', ascending=False)

**Сделайте выводы о качестве трёх рассмотренных моделей в данной задаче предсказания.**

Зависимость качества вина от остальных признаков, по-видимому, нелинейная. Поэтому Random Forest работает лучше в этой задаче.